# Problema
Una determinada empresa forestal puede producir $L$ productos distintos y tiene $I$ plantas productivas ubicadas en diferentes zonas, siendo $S_i^t$ la capacidad total de producción de la planta $i, (i=1,…,l)$ en el período $t, (t=1,…,T)$ sin importar de qué tipo de producto se trate, $T$ el número total de periodos. El tipo de producto $l$ tiene un costo de producción de $P_l$ sin importar la planta que lo fabrique ni el período en cuestión. Los productos son demandados por $J$ ciudades diferentes, siendo $D_{ljt}$ la demanda de la ciudad $j, (j=1,…,J)$ por el producto $l, (l=1,…,L)$, en el período $t, (t=1,…,T)$. Las demandas deben satisfechas período a período.

Como no existe la posibilidad de almacenar producto en las plantas, la empresa está estudiando la posibilidad de arrendar bodegas ubicadas en diferentes puntos geográficos. El arriendo de las bodegas se hace período a período, esto quiere decir que, si se arrienda la bodega $k$ en el período $t$, no necesariamente la bodega $k$ debe haber estado arrendada el período $t-1$ o seguir arrendada para el período $t+1$. Hay $K$ posibles bodegas para arrendar. De esta manera, la producción de las plantas se llevará a las bodegas y desde allí se abastecerá a las ciudades. No existe inventario de productos, las bodegas sólo se utilizan para etiquetar los distintos artículos. Si se arrienda la bodega $k,(k=1,…,K)$ se incurre en un gasto fijo $F_{kt}$ pesos por el pago del arriendo en el período $t$. La capacidad de la bodega $k$ es de ${Q_k}$ unidades de producto sin importar su tipo.

El costo de transporte del producto $l$ desde la planta $i$ a la bodega $k$ en el período $t$ es de $M_{likt}$ pesos y el costo de transporte desde la bodega $k$ a la ciudad $j$ del producto $l$ en el período $t$ es de $N_{lkjt}$ pesos.

# Modelo Matemático
## Conjuntos
$L$ = Conjunto de productos  
$I$ = Conjunto de plantas  
$T$ = Conjunto de periodos  
$J$ = Conjunto de ciudades  
$K$ = Conjunto de bodegas  

## Parametros
$S_i^t$ = Capacidad total de producción de la planta $i$ en el periodo $t$.  
$P_l$ = Costo de producción del producto $l$.  
$D_{ljt}$ = Demanda del producto $l$ de la ciudad $j$ en el periodo $t$.  
$F_{kt}$ = Costo fijo de arrendar la bodega $k$ en el periodo $t$.  
$Q_k$ = Capacidad de la bodega $k$.  
$M_{likt}$ = Costo de transporte del producto $l$ desde la planta $i$ a la bodega $k$ en el periodo $t$.  
$N_{lkjt}$ = Costo de transporte del producto $l$ desde la bodega $k$ a la ciudad $j$ en el periodo $t$.  

## Variables
$X_{likt}$ = Unidades del producto $l$ transportadas desde la planta $i$ a la bodega $k$ en el periodo $t$.  
$Y_{lkjt}$ = Unidades del producto $l$ transportadas desde la bodega $k$ a la ciudad $j$ en el periodo $t$.  
$\alpha_{kt}$ = $\begin{cases} 1 & \text{si la bodega $k$ se abre en el periodo $t$} \\ 0 & \text{~} \end{cases}$

## Modelo Matemático
\begin{equation}
\text{Min }Z = \sum_{l \in L} \sum_{i \in I} \sum_{k \in K} \sum_{t \in T} M_{likt} X_{likt} + \sum_{l \in L} \sum_{ k \in K} \sum_{j \in J} \sum_{t \in T} N_{lkjt} Y_{lkjt} + \\
\sum_{k \in K} \sum_{t \in T} F_{kt} a_{kt} + \sum_{l \in L} \sum_{i \in I} \sum_{k \in K} \sum_{t \in T} P_{l} X_{likt}
\end{equation}

\begin{align}
\text{subject to} & & \\
& \sum_{l \in L} \sum_{k \in K} X_{likt} \le S_{it} & \forall i \in I, t \in T \\
& \sum_{k \in K} Y_{lkjt} = D_{ljt} & \forall l \in L, j \in J, t \in T\\
& \sum_{l \in L} \sum_{i \in I} X_{likt} \le Q_{k} \alpha_{kt} & \forall k \in K, t \in T\\
& \sum_{i \in I} X_{likt} = \sum_{j \in J} Y_{lkjt} & \forall l \in L, k \in K, t \in T\\
& X_{likt} \ge 0 & \forall l \in L, i \in I, k \in K, t \in T \\
& Y_{lkjt} \ge 0 & \forall l \in L, k \in K, j \in J, t \in T \\
& \alpha_{kt} \in \{0,1\} & \forall k \in K, t \in T
\end{align}

# Implementación AMPL
```
reset;

param nproductos;
param nplantas;
param nperiodos;
param nciudades;
param nbodegas;

set L := {1..nproductos};
set I := {1..nplantas};
set T := {1..nperiodos};
set J := {1..nciudades};
set K := {1..nbodegas};

param S{I,T};
param P{L};
param D{L,J,T};
param F{K,T};
param Q{K};

param xcoordI{I} := round(Uniform(-100,100));
param ycoordI{I} := round(Uniform(-100,100));

param xcoordK{K} := round(Uniform(-100,100));
param ycoordK{K} := round(Uniform(-100,100));

param xcoordJ{J} := round(Uniform(-100,100));
param ycoordJ{J} := round(Uniform(-100,100));

# param M{L,I,K,T} default 10;
# param N{L,K,J,T} default 10;

param M{L, i in I, k in K,T} := 
            sqrt((xcoordI[i] - xcoordK[k])^2 + (ycoordI[i] - ycoordK[k])^2);
param N{L, k in K, j in J,T} := 
            sqrt((xcoordJ[j] - xcoordK[k])^2 + (ycoordJ[j] - ycoordK[k])^2);

var x{L,I,K,T} >= 0;
var y{L,K,J,T} >= 0;
var a{K,T} binary;

minimize z: sum{l in L, i in I, k in K, t in T} M[l,i,k,t]*x[l,i,k,t] + 
            sum{l in L, k in K, j in J, t in T} N[l,k,j,t]*y[l,k,j,t] + 
            sum{k in K, t in T} F[k,t]*a[k,t] + 
            sum{l in L, i in I, k in K, t in T} P[l]*x[l,i,k,t];

s.t.
    r1 {i in I, t in T}: sum{l in L, k in K} x[l,i,k,t] <= S[i,t];
    r2 {l in L, j in J, t in T}: sum{k in K} y[l,k,j,t] = D[l,j,t];
    r3 {k in K, t in T}: sum{l in L, i in I} x[l,i,k,t] <= Q[k]*a[k,t];
    r4 {l in L, k in K, t in T}: sum {i in I} x[l,i,k,t] = 
                                                sum {j in J} y[l,k,j,t];
    
data data.dat;
option solver cplex;
option cplex_options "mipdisplay=5";

solve;

for {l in L, i in I, k in K, t in T}{
    if x[l,i,k,t] >= 1 then
        printf "x[%d,%d,%d,%d] = %d\n",l,i,k,t,x[l,i,k,t];

}

for {l in L, k in K, j in J, t in T}{
    if y[l,k,j,t] >= 1 then
        printf "y[%d,%d,%d,%d] = %d\n",l,k,j,t,y[l,k,j,t];
}
```